In [ ]:
import keras
from keras.datasets import cifar10


In [ ]:
## cifar 10 데이터 로드
(x_train , y_train),(x_test, y_test)= cifar10.load_data()

import matplotlib.pyplot as plt
import numpy as np


In [ ]:

## cifar 10 dataset 시각화
fig = plt.figure(figsize=(20,5))
for i in range(36):
  ax = fig.add_subplot(3,12, i+1, xticks=[], yticks=[])
  ax.imshow(np.squeeze(x_train[i]))


In [ ]:
## 데이터 정규화
x_train = x_train.astype('float32')/ 255
x_test = x_test.astype('float32')/ 255


from keras.utils import to_categorical

## one-hot encoding
## cifar10 / class = 10 / 라벨이 10개
num_classes =  len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## valid 층을 train에서 따로 떼어 놓음 
(x_train, x_valid) = x_train[5000:], x_train[:5000]
(y_train, y_valid) = y_train[5000:], y_train[:5000]

print(x_train.shape,  x_valid.shape, x_test.shape)


In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout ,BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

## sequential 모델 사용
model= Sequential()
model.add(Conv2D(filters=16, kernel_size = 2, padding='same',activation= 'relu' , input_shape=(32,32,3)))
## 특징을 포함하되 사이즈를 반으로 
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size = 2, padding='same' ,activation= 'relu' ))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size = 2, padding='same',activation= 'relu' ))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10 ,activation='softmax' ))

model.summary()

## 학습전 컴파일 옵션 // metrics=['accuracy'] 는 분류 대상이 3개 이상일때 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
## val_loss가 더이상 떨어지지 않는 것이 10번  쌓이면 학습 중지
earlyStopping = EarlyStopping (monitor='val_loss', min_delta = 0, patience=10)
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit(x_train,y_train, batch_size=32, epochs=100, validation_data=(x_valid,y_valid), callbacks=[checkpointer, earlyStopping],verbose=2, shuffle=True)


In [ ]:

model2= Sequential()
model2.add(Conv2D(filters=32, kernel_size = 2, padding='same',activation= 'relu' , kernel_initializer='he_normal',input_shape=(32,32,3)))
## 배치 사이즈 정규화 실행
model2.add(BatchNormalization())
## 20프로의 뉴런 비활성화 --> 과적합 방지
model2.add(Dropout(0.2))
model2.add(Conv2D(filters=64, kernel_size = 3, padding='same' ,activation= 'relu' ,kernel_initializer='he_normal'))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=2))
model2.add(Conv2D(filters=128, kernel_size = 2, padding='same' ,activation= 'relu',kernel_initializer='he_normal' ))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=2))
model2.add(Conv2D(filters=64, kernel_size = 2, padding='same' ,activation= 'relu',kernel_initializer='he_normal' ))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=2))
model2.add(Dropout(0.2))

model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(10 ,activation='softmax' ,kernel_initializer='glorot_uniform'))

## 에폭마다 wb를 저장하기 위해
model2.load_weights('/content/model.weights.best.hdf5')

from keras.optimizers import SGD

model2.evaluate(x_test, y_test, verbose=0)


# compile model
opt = SGD(lr=0.001, momentum=0.9)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## callbacks=[checkpointer] 를 넣어서 에폭이 돌때마다 WB 저장
hist = model2.fit(x_train,y_train, batch_size=64, epochs=100, validation_data=(x_valid,y_valid), callbacks=[checkpointer],verbose=2, shuffle=True)

print(hist.history['loss'])